In [2]:
#env = gym_tetris.make('TetrisA-v2',deterministic = True)
#env = JoypadSpace(env, MOVEMENT)
#env.reset()

In [3]:
#env.render()

In [4]:
print(MOVEMENT)

[['NOOP'], ['A'], ['B'], ['right'], ['right', 'A'], ['right', 'B'], ['left'], ['left', 'A'], ['left', 'B'], ['down'], ['down', 'A'], ['down', 'B']]


In [1]:
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT
import time


def ColorBoardtoSimpleBoard(board):
    simpleboard = []
    for row in board:
        simplerow = []
        for cell in row:
            if cell == 239:
                simplerow.append(0)
            else:
                simplerow.append(1)
        simpleboard.append(tuple(simplerow))
    return tuple(simpleboard)

class Tester:
    def __init__(self, episodes=100):
        self.env = gym_tetris.make('TetrisA-v2')
        self.env = JoypadSpace(self.env, MOVEMENT)
        #self.env.deterministic = True
        #Testing to see whether using the pixels for the state works better than just the board.
        #self.state = self.env.reset()
        self.env.reset()
        #self.env.deterministic = True
        self.env.render()
        self.state = tuple([ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), "", "", tuple(self.env.ram[0x0040:0x0042])])
        #print(self.env.ram[0x0044])
        
        self.piecesChecked = []
        self.DownLeft = 0
        self.info = "NONE" 
        self.wait = 5
        
        self.actions = MOVEMENT
        self.state_actions = []  # state & action track

        self.episodes = episodes  # number of episodes going to play
        self.steps_per_episode = []
        
        
        
    def chooseAction(self):
        action = []
        #Insert macro/micro algorithm here:
        
        #Right now just have the agent make random actions until we get the 2-stage algorithm working
        #print(self.actions)
        if self.DownLeft == 1:
            action.append("down")
        elif self.wait <= 0:
            action.append("left")
        if self.info in self.piecesChecked:
            action.append("A")
        if len(action) == 0:
            action.append("NOOP")
            
        self.wait -= 1
        #print(action)
        
        return self.actions.index(action)
    
    def reset(self):
        self.env.close()
        self.env = gym_tetris.make('TetrisA-v2')
        self.env = JoypadSpace(self.env, MOVEMENT)
        #self.env.deterministic = True
        self.env.reset()
        #self.env.deterministic = True
        self.env.render()
        self.info = "NONE"
        self.wait = 100
        self.state = tuple([ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), "", "", tuple(self.env.ram[0x0040:0x0042])])
        #Conversion of self.state to tuple for hashing purposes
        #self.state = tuple([tuple(x) for x in self.state])
        self.state_actions = []


    def play(self):
        self.steps_per_episode = []  
        
        for ep in range(self.episodes):
            done = False
            while not done:
                self.DownLeft += 1
                if self.DownLeft > 1:
                    self.DownLeft = 0
                action = self.chooseAction()
                #print(action)
                self.state_actions.append((self.state, action))
                try:
                    self.info = info["current_piece"]
                    self.prevpos = tuple(self.env.ram[0x0040:0x0042])
                except:
                    j = "j"

                #unusedstate, reward, done, info = self.env.step(self.env.action_space.sample())
                #print(self.env.action_space.sample())
                unusedstate, reward, done, info = self.env.step(action)
                self.env.render()
                if self.info[0] != "J" and self.info != "NONE":
                    info = None
                    self.reset()
                    continue
                #print(self.env.ram[0x0045])
                if self.info[0] != info["current_piece"][0] and self.info != "NONE":
                    print(self.info)
                    print(self.prevpos)
                    if self.info not in self.piecesChecked:
                        self.piecesChecked.append(self.info)
                        print(self.piecesChecked)
                        time.sleep(10)
                    info = None
                    self.reset()
                    continue
                
                
                nxtState = tuple([ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()), info["current_piece"], info["next_piece"], tuple(self.env.ram[0x0040:0x0042])])

            # end of game
            #if ep % 10 == 0:
            #print(ColorBoardtoSimpleBoard(self.env.ram[0x0400:0x04C8].reshape((20, 10)).copy()))
            #self.reset()
        self.env.close()

if __name__ == "__main__":
    N_EPISODES = 1
    # comparison
    agent = Tester(episodes=N_EPISODES)
    agent.play()

C:\Users\nickg\anaconda3\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment TetrisA-v2 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
C:\Users\nickg\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
C:\Users\nickg\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(
C:\Users\nickg\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to re

Jd
(1, 18)
['Jd']
Jl
(1, 18)
['Jd', 'Jl']


KeyboardInterrupt: 

In [ ]:
time.sleep(10)
print("E")

In [12]:
print(agent.env._action_map)

{0: 0, 1: 1, 2: 2, 3: 128, 4: 129, 5: 130, 6: 64, 7: 65, 8: 66, 9: 32, 10: 33, 11: 34}
